In [1]:
import os, shutil, sys
import pandas as pd
import numpy as np
import ezc3d
import matplotlib.pyplot as plt
import matplotlib.image as image

user = os.getcwd().split('\\')[os.getcwd().split('\\').index('Users') + 1]

# fork https://github.com/kww-22/auto-marker-label first
github_path = r"c:\users\{}\documents\github\auto-marker-label".format(user)
sys.path.append(github_path)
from automarkerlabel import marker_label, import_raw_c3d, import_markerSet

data_path = os.path.join(github_path, 'dl_data')
c3d_path = os.path.join(data_path, 'test')
modelfiles = os.listdir(c3d_path)
modelfiles = [f for f in modelfiles if f.endswith('.c3d')]

In [3]:
for file in modelfiles:
    c = import_raw_c3d(os.path.join(c3d_path, file), 0)
    pts = c[0]
    fs_point = c[1]
    old_labels = c[2]
    windowSize  = int(fs_point*0.5)
    modelpath =  os.path.join(data_path, 'model_tpose.ckpt') # need to unzip before running notebook
    trainvalpath = os.path.join(data_path, 'trainingvals_model_tpose.pickle')
    markersetpath = os.path.join(data_path, 'MarkerSet_DL.xml')
    markers, segment, uniqueSegs, segID, mkcoordL, num_mks = import_markerSet(markersetpath)

    new_labels, confidence, Y_pred = marker_label(pts, modelpath, trainvalpath, markersetpath, fs_point, windowSize) # where the magic happens

    # export marker label key
    csv_exp = pd.DataFrame([old_labels, new_labels, confidence]).T
    csv_exp.columns = ['old_labels', 'new_labels', 'confidence']
    csv_exp.to_csv(os.path.join(c3d_path, file.split('.')[0] + '_key.csv'), index=False)
    
    # export labelled c3d
    # export_labelled_c3d(pts, d[0], 0, os.path.join(c3d_path, file), os.path.join(c3d_path, file.split('.')[0] + '_labelled.c3d'), markers, 0)
